In [1]:
import pandas as pd
import openpyxl
import warnings
import os
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font, numbers
from openpyxl.utils import get_column_letter

warnings.filterwarnings("ignore")

base_path = "BASE.xlsx"
resultado = "resultado.xlsx"
dacxanalista_path = "Nuevo_DACxANALISTA.xlsx"

df_base = pd.read_excel(base_path)
df_base = df_base.iloc[:, 3:] # Eliminar columnas iniciales
df_base = df_base.iloc[7:, :] # Eliminar filas iniciales
df_base = df_base.rename(columns=df_base.iloc[0]) # Renombra las columnas
df_base = df_base[1:]
df_base = df_base.reset_index(drop=True) # Reinicia los indices
df_base = df_base.dropna(subset=["Cuenta","ACC"]) # Elimina filas con valores NaN
df_base = df_base.reset_index(drop=True)
columnas_deseadas = ["ACC", "Cuenta", "Demora", "     Importe en ML"]
df_base = df_base[columnas_deseadas]
df_base = df_base.rename(columns={"     Importe en ML": "Importe"})
df_base["Demora"] = df_base["Demora"].astype("Int64")
df_base["Importe"] = df_base["Importe"].astype(float)
# Condition 1
df_base["Status"] = df_base["Importe"].apply(lambda x: "DEUDA" if x > 0 else "SALDOS A FAVOR")
# Condition 2
df_base["Tipo Deuda"] = df_base["Demora"].apply(lambda x: "CORRIENTE" if x <= 0 else "VENCIDA")
# Condition 3
df_base["Saldo Final"] = df_base.apply(lambda row: row["Importe"] if (row["Status"] == "DEUDA" and row["Tipo Deuda"] == "VENCIDA") else (row["Importe"] if row["Status"] == "SALDOS A FAVOR" else "NO"), axis=1)
df_base = df_base[df_base["Saldo Final"] != "NO"]
df_base = df_base.sort_values(by=["Cuenta"], ascending=[True])
df_base = df_base.sort_values(by=["ACC"], ascending=[True])
df_base = df_base.sort_values(by=["Demora"], ascending=[False])
df_base = df_base.reset_index(drop=True)

ultima_fila = df_base.shape[0]
for i in range(ultima_fila):
    if df_base.loc[i, "Status"] == "DEUDA":
        saldoDeuda = df_base.loc[i, "Saldo Final"]
        for j in range(ultima_fila):
            if (
                df_base.loc[i, "Cuenta"]    == df_base.loc[j, "Cuenta"] and 
                df_base.loc[i, "ACC"]       == df_base.loc[j, "ACC"]    and 
                df_base.loc[j, "Status"]    == "SALDOS A FAVOR"
                ):
                saldoFavor = df_base.loc[j, "Saldo Final"]
                montoCompensar = min(saldoDeuda, abs(saldoFavor))
                df_base.loc[i, "Saldo Final"] = saldoDeuda - montoCompensar
                df_base.loc[j, "Saldo Final"] = saldoFavor + montoCompensar
                saldoDeuda = df_base.loc[i, "Saldo Final"]

df_base = df_base[(df_base["Tipo Deuda"] == "VENCIDA") & (df_base["Status"] == "DEUDA")]
df_base = df_base.reset_index(drop=True)

grouped_df = df_base.groupby(["Cuenta", "ACC"]).agg({"Demora": "max", "Saldo Final": "sum"})

df_final = grouped_df.reset_index()[["Cuenta", "ACC", "Saldo Final", "Demora"]]
df_final = df_final.rename(columns={"Cuenta":"Cod Cliente", "ACC":"Área Ctrl", "Saldo Final":"Deuda Vencida", "Demora":"Días de Morosidad"})

df_dacxanalista = pd.read_excel(dacxanalista_path, sheet_name="Base_NUEVA")

df_final = df_final.merge(df_dacxanalista[["DEUDOR", "NOMBRE"]], left_on="Cod Cliente", right_on="DEUDOR", how="left")
df_final = df_final.rename(columns={"NOMBRE": "Razón Social"})
df_final = df_final.drop(columns=["DEUDOR"])

areas_de_control = {
    "PE01": "Post-Pago",
    "PE02": "Pre-Pago",
    "PE03": "Tiempo Aire",
    "PE04": "Reintegro",
    "PE05": "Reestructura",
    "PE07": "Contado / Administrativas",
    "PE09": "Cargos Admtivos / Otros",
    "PE10": "Sim Card",
    "PE11": "Recarga Prepago",
    "PE12": "Recarga Física",
    "PE13": "Arrendamiento",
    "PE14": "Tel.Fija Inalamb.",
    "PE15": "Prendas",
    "PE16": "DTH",
    "PE17": "HFC"
}
df_final["Producto"] = df_final["Área Ctrl"].apply(lambda x: areas_de_control[x])
df_final["Código Pago"] = "33" + df_final["Área Ctrl"].str[-2:] + df_final["Cod Cliente"].astype(str)
df_final = df_final[["Cod Cliente", "Razón Social", "Área Ctrl", "Producto", "Deuda Vencida", "Código Pago", "Días de Morosidad"]]

df_final.to_excel(resultado, index=False)

os.startfile(resultado)
print(df_final.shape)
df_final.head(20)

(80, 7)


,Cod Cliente,Razón Social,Área Ctrl,Producto,Deuda Vencida,Código Pago,Días de Morosidad
0,4070150,FENIX COMUNICACIONES EIRL,PE01,Post-Pago,1051.72,33014070150,410
1,4070150,FENIX COMUNICACIONES EIRL,PE16,DTH,215889.65,33164070150,1146
2,4070209,CONSULTORIA EMPRESARIAL PALACIOS TORRES & ASOC...,PE10,Sim Card,693.84,33104070209,1
3,4070435,LAULATE ACHO CARMEN DEL PILAR,PE02,Pre-Pago,230.02,33024070435,6
4,4070587,INTEL PHONE SAC.,PE01,Post-Pago,1525.5,33014070587,2
5,4070587,INTEL PHONE SAC.,PE02,Pre-Pago,17.15,33024070587,6
6,4070649,DT PROYECTOS S.A.C,PE16,DTH,59000.0,33164070649,6
7,4070880,LUVITEL E.I.R.L,PE02,Pre-Pago,0.0,33024070880,6
8,4070880,LUVITEL E.I.R.L,PE03,Tiempo Aire,0.0,33034070880,3
9,4070880,LUVITEL E.I.R.L,PE07,Contado / Administrativas,0.0,33074070880,5
